In [1]:
import numpy as np
import json
import os
from pathlib import Path

import PySAM.Utilityrate5 as ur
import PySAM.Battery as stbt
import PySAM.Cashloan as loan
import PySAM.Pvwattsv8 as pvwatts

file_dir = os.path.abspath("")
output_dir = file_dir + "sam_results_6"
key = "[TODO:replace]"

#reopt_api_post_url = 'http://localhost:8000/stable/job?format=json'
#reopt_api_poll_url = 'http://localhost:8000/stable/job/'

reopt_api_post_url = 'https://developer.nrel.gov/api/reopt/v1/job?format=json'
reopt_api_poll_url = 'https://developer.nrel.gov/api/reopt/v1/job/'


In [2]:
# Translate from the V1 API given by pysam to the V3 structure used by Julia
def translated_reopt_post(reopt_post):
    scenario = reopt_post["Scenario"]
    site = scenario["Site"]

    v3_format = {}
    v3_format["Site"] = {"latitude" : site["latitude"], "longitude" : site["longitude"]}
    v3_format["Storage"] = site["Storage"]
    v3_format["Storage"]["installed_cost_per_kw"] = site["Storage"]["installed_cost_us_dollars_per_kw"]
    v3_format["Storage"]["installed_cost_per_kwh"] = site["Storage"]["installed_cost_us_dollars_per_kwh"]
    v3_format["Storage"]["replace_cost_per_kwh"] = site["Storage"]["replace_cost_us_dollars_per_kwh"]
    v3_format["Storage"].pop("installed_cost_us_dollars_per_kw", None)
    v3_format["Storage"].pop("installed_cost_us_dollars_per_kwh", None)
    v3_format["Storage"].pop("replace_cost_us_dollars_per_kwh", None)
    v3_format["PV"] = site["PV"]
    v3_format["PV"]["production_factor_series"] = np.array(site["PV"]["prod_factor_series_kw"]).tolist()
    v3_format["PV"].pop("prod_factor_series_kw", None)
    #v3_format["PV"]["installed_cost_per_kw"] = site["PV"]["installed_cost_us_dollars_per_kw"]
    v3_format["Financial"] = site["Financial"]
    v3_format["Financial"]["analysis_years"] = int(site["Financial"]["analysis_years"])
    v3_format["Financial"]["value_of_lost_load_per_kwh"] = site["Financial"]["value_of_lost_load_us_dollars_per_kwh"]
   # v3_format["Financial"]["om_cost_escalation_rate_fraction"] = site["Financial"]["om_cost_escalation_pct"] # TODO: these are seperate in SAM, seperate them for final code
    v3_format["Financial"]["elec_cost_escalation_rate_fraction"] = site["Financial"]["escalation_pct"]
    v3_format["Financial"]["offtaker_tax_rate_fraction"] = site["Financial"]["offtaker_tax_pct"]
    v3_format["Financial"]["offtaker_discount_rate_fraction"] = site["Financial"]["offtaker_discount_pct"]
    v3_format["Financial"]["microgrid_upgrade_cost_fraction"] = site["Financial"]["microgrid_upgrade_cost_pct"]
    v3_format["Financial"].pop("value_of_lost_load_us_dollars_per_kwh", None)
    v3_format["Financial"].pop("escalation_pct", None)
    v3_format["Financial"].pop("om_cost_escalation_pct", None)
    v3_format["Financial"].pop("offtaker_tax_pct", None)
    v3_format["Financial"].pop("offtaker_discount_pct", None)
    v3_format["Financial"].pop("microgrid_upgrade_cost_pct", None)
    v3_format["ElectricLoad"] = site["LoadProfile"]
    v3_format["ElectricLoad"]["loads_kw"] = np.array(v3_format["ElectricLoad"]["loads_kw"]).tolist()
    v3_format["ElectricLoad"]["critical_loads_kw"] = np.array(v3_format["ElectricLoad"]["critical_loads_kw"]).tolist()
    v3_format["ElectricLoad"]["loads_kw_is_net"] = bool(v3_format["ElectricLoad"]["loads_kw_is_net"] == 1.0 or v3_format["ElectricLoad"]["loads_kw_is_net"])
    v3_format["ElectricTariff"] = site["ElectricTariff"]
    for k, v in v3_format["ElectricTariff"]["urdb_response"].items():
        if type(v) is tuple:
            if type(v[0]) is tuple:
                arr = []
                for i, n in enumerate(v):
                    
                    if type(n[0]) is dict:
                        for m in n:
                            if "max" in m:
                                if m["max"] > 1e15:
                                   m["max"] = 1e15
                            
                        arr.append(np.array(n, dtype=dict))
                        
                        #if "flatdemandstructure" in k:
                            # Append extra element to trick REopt
                        #    arr.append(np.array(n, dtype=dict).tolist())

                    elif "schedule" in k:
                        arr.append(np.array(n, dtype=int))
                    else:
                        arr.append(np.array(n))
                v3_format["ElectricTariff"]["urdb_response"][k] = np.array(arr)
            else:
                if "flatdemandmonths" in k:
                    v3_format["ElectricTariff"]["urdb_response"][k] = np.array(v, dtype=int)
                else:
                    v3_format["ElectricTariff"]["urdb_response"][k] = np.array(v)

    return v3_format


In [3]:
# ipython julia interpreter so we can debug the function here rather than in reopt.jl
%load_ext julia.magic


Initializing Julia interpreter. This may take some time...


In [4]:
from julia import Main

weather_file = file_dir + "/USA_CA_San.Diego.Lindbergh.722900_2018.epw"

pv = pvwatts.default("PVWattsCommercial")
pv.SolarResource.solar_resource_file = weather_file
pv.SolarResource.use_wf_albedo = 0
pv.Lifetime.dc_degradation = [0.005]


bt = stbt.from_existing(pv, "GenericBatteryCommercial")
bt.Load.crit_load = [0] * 8760
fin = ur.from_existing(bt, "GenericBatteryCommercial")
cl = loan.from_existing(bt, "GenericBatteryCommercial")

pv.execute()
post = pv.Reopt_size_battery_post()

reopt_post = post["reopt_post"]
site = reopt_post["Scenario"]["Site"]
site["latitude"] = 32.5
site["longitude"] = -108

storage = site["Storage"]
storage["installed_cost_us_dollars_per_kw"] = 405.56
storage["installed_cost_us_dollars_per_kwh"] = 225.06

reopt_pv = {}
reopt_pv["inv_eff"] = 0.96
reopt_pv["dc_ac_ratio"] = 1.2
reopt_pv["losses"] = 0.0
reopt_pv["prod_factor_series_kw"] = pv.Outputs.gen

site["PV"] = reopt_pv
reopt_post["Scenario"]["Site"] = site

reopt_json = translated_reopt_post(reopt_post)

Main.eval('using Pkg; Pkg.add("JuMP"); Pkg.add("HiGHS"); Pkg.add("REopt"); Pkg.add("JSON")')
Main.eval('using JuMP; using HiGHS; using JSON; using REopt')
Main.data = reopt_json 
print("Python: list of list of dicts: ", reopt_json["ElectricTariff"]["urdb_response"]["flatdemandstructure"])
Main.model = Main.eval('Model(HiGHS.Optimizer)')
Main.tarriff = Main.eval('data["ElectricTariff"]')
Main.urdb = Main.eval('tarriff["urdb_response"]')
Main.flat_demand = Main.eval('urdb["flatdemandstructure"]')

#Main.eval('print(urdb)')
print("Julia: list of dicts")
Main.eval('print(flat_demand)')
Main.eval('\n')

    Updating registry at `C:\Users\wbecker\.julia\registries\General`

JuliaError: Exception 'could not load library "libgit2"
The specified module could not be found. ' occurred while calling julia code:
using Pkg; Pkg.add("JuMP"); Pkg.add("HiGHS"); Pkg.add("REopt"); Pkg.add("JSON")

In [5]:
# Function from urdb.jl here for debugging. Only difference is added print statements
Main.eval('\
function parse_urdb_demand_tiers(A::Array; bigM=1.0e8) \n\
    if length(A) == 0 \n\
        return [] \n\
    end \n\
    len_tiers = Int[length(r) for r in A] \n\
    print(A) \n\
    n_tiers = maximum(len_tiers) \n\
    period_with_max_tiers = findall(len_tiers .== maximum(len_tiers))[1] \n\
\
    # set up tiers and validate that the highest tier has the same value across periods \n\
    demand_tiers = Dict() \n\
    demand_maxes = Float64[] \n\
    for period in range(1, stop=length(A)) \n\
        demand_max = Float64[] \n\
        for tier in A[period] \n\
            append!(demand_max, get(tier, "max", bigM)) \n\
        end \n\
        demand_tiers[period] = demand_max \n\
        append!(demand_maxes, demand_max[end])  # TODO should this be maximum(demand_max)? \n\
    end \n\
\
    # test if the highest tier is the same across all periods \n\
    if length(Set(demand_maxes)) > 1 \n\
        @warn "Highest demand tiers do not match across periods: using max tier from largest set of tiers." \n\
    end \n\
    return demand_tiers[period_with_max_tiers] \n\
end\
')
Main.tiers = Main.parse_urdb_demand_tiers(Main.flat_demand)
print(Main.tiers)

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


: 

: 

In [ ]:
from julia import Julia

jl = Julia(debug=True)

from julia import Main

weather_file = file_dir + "/e_plus_loads/USA_CA_San.Diego.Lindbergh.722900_2018.epw"

pv = pvwatts.default("PVWattsCommercial")
pv.SolarResource.solar_resource_file = weather_file
pv.execute()

bt = stbt.default("GenericBatteryCommercial")
fin = ur.from_existing(bt, "GenericBatteryCommercial")
cl = loan.from_existing(bt, "GenericBatteryCommercial")

bt.Load.crit_load = [0] * 8760
post = bt.Reopt_size_standalone_battery_post()

reopt_post = post["reopt_post"]
site = reopt_post["Scenario"]["Site"]
site["latitude"] = 32.5
site["longitude"] = -108

storage = site["Storage"]
storage["installed_cost_us_dollars_per_kw"] = 405.56
storage["installed_cost_us_dollars_per_kwh"] = 225.06

reopt_pv = {}
reopt_pv["inv_eff"] = 0.96
reopt_pv["dc_ac_ratio"] = 1.2
reopt_pv["losses"] = 0.0
reopt_pv["prod_factor_series_kw"] = pv.Outputs.gen

site["PV"] = reopt_pv
reopt_post["Scenario"]["Site"] = site

reopt_json = translated_reopt_post(reopt_post)
outfile = open("reopt_data.json", "w")
json.dump(reopt_json, outfile, indent=4)
outfile.close()

data_string = json.dumps(reopt_json)

Main.eval('using Pkg; Pkg.add("JuMP"); Pkg.add("HiGHS"); Pkg.add("REopt"); Pkg.add("JSON")')
Main.eval('using JuMP; using HiGHS; using JSON; using REopt')
Main.data_string = data_string
Main.model = Main.eval('Model(HiGHS.Optimizer)')
Main.eval('data = JSON.parse(data_string)')
Main.s = Main.Scenario(Main.data)
Main.eval('inputs = REoptInputs(s)')
Main.eval('results=run_reopt(model, inputs)')
Main.results